# Domain adaptive Pretraining
#### Language: `Arabic`

Continual pretraining on the unlablled translated corpus to improve the performance of the model.

In [11]:
import sys
sys.path.append('../')
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import json # to read json
import warnings # to ignore warnings
from utils import squad_json_to_dataframe_train
warnings.filterwarnings('ignore')

In [12]:
# training data
language = 'ar'
input_file_path = f'../SQuAD/translate-train/squad.translate.train.en-{language}.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path,verbose=0)

In [13]:
train.head()

,index,question,context,answer_start,text,c_id
0,5706071252bb8914006897cd,أين تزود المهاجرين بالوقود؟,بالنسبة لبعض أنواع الخواضون ، يعتمد نجاح الهجر...,86,نقاط التوقف على طول طريق الترحيل,0
1,56f82171a6d7ea1400e173e9,من المشهور الذي عبر جبال الألب مع قطيع من الفيلة؟,اكتشف رجل محنط ، يبلغ عمره 5000 عام ، على نهر ...,165,هانيبال,1
2,5726c1e7708984140094d0a2,متى غادر الرومان أوترخت,من منتصف القرن الثالث غزت القبائل الجرمانية با...,71,حوالي 275 لم يعد بإمكان الرومان الحفاظ على الح...,2
3,570e6b020b85d914000d7eb9,ما هي النظرية المسماة التعامل مع نقل السنسكريت...,من أجل شرح الميزات الشائعة التي تتقاسمها اللغة...,118,نظرية هجرة الهند الآرية,3
4,5733cf6a4776f419006612a4,ماذا تمتلك المعاهدة التي يفتقر إليها الآخرون؟,في عام 1998 ، صرح كورت جوناسون وكارين بيورنسون...,262,مصداقية قانونية دولية,4


In [14]:
train.drop(["index","answer_start","text","c_id"],axis = 1,inplace=True)
train.head()

,question,context
0,أين تزود المهاجرين بالوقود؟,بالنسبة لبعض أنواع الخواضون ، يعتمد نجاح الهجر...
1,من المشهور الذي عبر جبال الألب مع قطيع من الفيلة؟,اكتشف رجل محنط ، يبلغ عمره 5000 عام ، على نهر ...
2,متى غادر الرومان أوترخت,من منتصف القرن الثالث غزت القبائل الجرمانية با...
3,ما هي النظرية المسماة التعامل مع نقل السنسكريت...,من أجل شرح الميزات الشائعة التي تتقاسمها اللغة...
4,ماذا تمتلك المعاهدة التي يفتقر إليها الآخرون؟,في عام 1998 ، صرح كورت جوناسون وكارين بيورنسون...


In [16]:
pretraining_data_path = f"./squad.pretraining.{language}.csv"
train.to_csv(pretraining_data_path,index=False)

In [17]:
from transformers import AutoTokenizer, LineByLineTextDataset

# model_name = "microsoft/Multilingual-MiniLM-L12-H384"
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
dataset = LineByLineTextDataset(tokenizer=tokenizer, file_path=input_file_path, block_size=128)

In [ ]:
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True, mlm_probability=0.15)    

 
training_args = TrainingArguments(
    output_dir="twimbert",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=16,
    save_total_limit=1,
)